# Building the book

In [41]:
from IPython.display import Code

## Compilation

In the Jupyter interface, open the terminal (`File > New > Terminal`) and run  
```bash
./build.sh
```
under the *repository root*. Answer `y` or simply press enter when prompted.

To preview the book website,
- start the HTTP server (`File > New Launcher > www`) and 
- navigate to `www/cs5483jupyter`.

The source code of `build.sh` is shown below:

In [4]:
Code('../build.sh')

#!/bin/bash

cmds=(
    "jupyter-book clean docs"
    "jupyter-book build docs"
    "rm -rf \"${JUPYTER_SERVER_ROOT}/www/cs5483jupyter\" && mkdir --parents \$_ && cp -rf docs/_build/html/* \$_"
    ) 

if [ -z ${JUPYTER_SERVER_ROOT} ]
then

    echo "[Error] Must run in a jupyter environment." 1>&2
    exit 1

else

    for cmd in "${cmds[@]}"
    do
        read -r -p "${cmd}?[Y/n] " input

        case $input in
            [yY][eE][sS]|[yY]|'')
        echo "Executing..."
        eval $cmd
        ;;
            [nN][oO]|[nN])
        echo "Skipped..."
            ;;
            *)
        echo "Invalid input..."
        exit 1
        ;;
        esac
    done

fi

- `jupyter-book clean docs` to empty previously built book.
- `jupyter-book build docs` to build the book.
- `rm -rf \"${JUPYTER_SERVER_ROOT}/www/cs5483jupyter\" && mkdir --parents \$_ && cp -rf docs/_build/html/* \$_` moves the book to the folder `www/cs5483jupyter` under the Jupyter Server Root for the HTTP server to serve the book website.

```{tip}
- You can run `yes | ./build.sh` to automatically answer `y` when prompted.
- After rebuilding the book, you can refresh an existing preview page to update its content.
```

```{seealso}
- [JupyterBook docs on CLI reference](https://jupyterbook.org/reference/cli.html)
```

## Configuration

If you use this book as a template in your own repository to host the book, you should edit the configuration file `_config.yml` under the `docs` folder:

In [42]:
Code('_config.yml')

#######################################################################################
# Book settings
title                       : Jupyter for CS5483
author                      : Chung Chan  # The author of the book
copyright                   : "2021"      # Copyright year to be placed in the footer
logo                        : logo.dio.svg  # A path to the book logo
# Patterns to skip when building the book. Can be glob-style (e.g. "*skip.ipynb")
exclude_patterns            : [_build, Thumbs.db, .DS_Store, "**.ipynb_checkpoints"]
# Auto-exclude files not in the toc
only_build_toc_files        : true
# Add a bibtex file so that we can create citations
bibtex_bibfiles:
  - ref.bib

#######################################################################################
# Execution settings
execute:
  execute_notebooks         : auto  # Whether to execute notebooks at build time. Must be one of ("auto", "force", "cache", "off")
  cache                     : ""    # A path to the jupyter cache that will be used to store execution artifacts. Defaults to `_build/.jupyter_cache/`
  exclude_patterns          : []    # A list of patterns to *skip* in execution (e.g. a notebook that takes a really long time)
  timeout                   : 30    # The maximum time (in seconds) each notebook cell is allowed to run.
  run_in_temp               : false # If `True`, then a temporary directory will be created and used as the command working directory (cwd),
                                    # otherwise the notebook's parent directory will be the cwd.
  allow_errors              : true  # If `False`, when a code cell raises an error the execution is stopped, otherwise all cells are always run.
  stderr_output             : show  # One of 'show', 'remove', 'remove-warn', 'warn', 'error', 'severe'


#######################################################################################
# Parse and render settings
parse:
  myst_url_schemes          : [mailto, http, https]  # URI schemes that will be recognised as external URLs in Markdown links
  myst_enable_extensions    :    
    # - amsmath
    - colon_fence
    # - deflist
    - dollarmath
    # - html_admonition
    # - html_image
    - linkify
    # - replacements
    # - smartquotes
    - substitution
  myst_dmath_allow_labels   : true
  myst_dmath_double_inline  : true  # To allow display math (i.e. $$) within an inline context

#######################################################################################
# HTML-specific settings
html:
  favicon                   : ""  # A path to a favicon image
  use_edit_page_button      : true  # Whether to add an "edit this page" button to pages. If `true`, repository information in repository: must be filled in
  use_repository_button     : true  # Whether to add a link to your repository button
  use_issues_button         : true  # Whether to add an "open an issue" button
  extra_navbar              : Powered by <a href="https://jupyterbook.org">Jupyter Book</a>  # Will be displayed underneath the left navbar.
  extra_footer              : ""  # Will be displayed underneath the footer.
  google_analytics_id       : ""  # A GA id that can be used to track book views.
  home_page_in_navbar       : true  # Whether to include your home page in the left Navigation Bar
  baseurl                   : ""  # The base URL where your book will be hosted. Used for creating image previews and social links. e.g.: https://mypage.com/mybook/
  comments:
    hypothesis              : true
    utterances              : false

#######################################################################################
# LaTeX-specific settings
latex:
  latex_engine              : xelatex  # one of 'pdflatex', 'xelatex' (recommended for unicode), 'luatex', 'platex', 'uplatex'
  use_jupyterbook_latex     : true # use sphinx-jupyterbook-latex for pdf builds as default
  
#######################################################################################
#

In particular, you should change the repository url is specified by the lines:

```yaml
repository:
  url: "https://github.com/ccha23/cs5483jupyter"  # Online location of your book
```

to your repository while retaining `github.com` as the base url, i.e.,

```yaml
repository:
  url: "https://github.com/{repo path}"  # Online location of your book
```

where the repository is actually hosted at `https://gitlab1.cs.cityu.edu.hk/{repo path}`.

```{important}
This is a hack to [support GitLab pages](https://github.com/executablebooks/jupyter-book/issues/1416):
- Using `github.com` tricks `jupyter-book` to enable the launch buttons and other repository-related buttons.
- The javascript `github2gitlab.js` under the folder `_static` then replaces `github.com` in the buttons by `gitlab1.cs.cityu.edu.hk`.

If you host your repository in other git servers, you should also edit `github2gitlab.js` to substitute `gitlab1.cs.cityu.edu.hk` with the appropriate server name.
```

In [45]:
Code('_static/github2gitlab.js')

window.addEventListener("load", 
    function () {
        $("a.binder-button").map(function () {
            re = /(https:\/\/mybinder\.org.*\/)gh\/(.*\/)(.*)\/(.*)\?(urlpath=lab\/tree\/)(.*)/
            this.href = this.href.replace(re, function (match, p1, p2, p3, p4, p5, p6, offset, string) {
                p2_ = ['https://gitlab1.cs.cityu.edu.hk', '/', p2, p3].join('');
                return [p1, 'git/', encodeURIComponent(p2_), '/', p4, '?', 'urlpath=git-pull?', 
                    encodeURIComponent(['repo=', p2_, '&branch=', p4,'&', p5, p3, '/',  p6].join(''))].join('');
            })
        });
        $("a.jupyterhub-button").map(function () {
            re = /(https:\/\/.*)https:\/\/github\.com(\/.*)/
            this.href = this.href.replace(re, function (match, p1, p2, offset, string) {
                return [p1, 'https://gitlab1.cs.cityu.edu.hk', p2].join('');
            })
        });
        $("a.repository-button,a.issues-button,a.edit-button").map(function () {
            re = /https:\/\/github\.com(\/.*)/
            this.href = this.href.replace(re, function (match, p1, offset, string) {
                return ['https://gitlab1.cs.cityu.edu.hk', p1].join('');
            })
        });
        $(".dropdown-buttons-trigger i.fa-github").addClass('fa-gitlab').removeClass('fa-github');
    }
)

```{seealso}
- [JupyterBook docs on Config reference](https://jupyterbook.org/customize/config.html)
```